*Enter your observations here*

In [1]:
import mailbox
import csv
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import re
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA








stop_words = set(stopwords.words('english'))
print(stop_words)

def createCSV(name, mbox, mes_type):
    #createing SPAM csv
    writer = csv.writer(open(name, "w", encoding="utf-8"))
    writer.writerow(['Subject', 'From', 'Date', 'Message-ID', 'Payload', 'Type'])
    for message in mailbox.mbox(mbox):

        if message.is_multipart():
            content = ''.join(str(part.get_payload(decode=True)) for part in message.get_payload())
            content = cleanMe(content)
        else:
            content = str(message.get_payload(decode=True))
            content = cleanMe(content)
#         content = content.split()
        #print(content)
        writer.writerow([message['subject'], message['from'], message['date'], message['Message-Id'], content, mes_type])
    

{"should've", 's', 'at', 'in', 'above', 'ma', 'm', 'should', 'didn', 'yours', 'the', "hasn't", 'did', 'needn', 'we', 'those', "you'd", 'hers', 'won', 'ours', 'mustn', 'was', "you'll", 'isn', 'she', 'and', 'if', 'you', 'herself', "weren't", 'what', 'who', 'any', 'haven', 'mightn', 'down', 'each', 'further', 'but', 'most', 'yourselves', 'his', 't', 'between', 'again', 'ourselves', 'an', 'other', "hadn't", "it's", "aren't", 'these', 'shan', 'some', 'y', 'have', 'were', 'by', 'he', 'had', 'until', 'of', 'nor', 'himself', "you've", 'their', "didn't", 'myself', 'that', 'your', 'here', 'does', 'under', "isn't", "mightn't", 'up', 'themselves', 'where', 'how', 'shouldn', 'now', "doesn't", 'be', 'has', 'hasn', 'no', 'will', 'when', 'doing', 'below', 'or', 'for', 'there', 'from', 'then', 'me', 'whom', "she's", 'ain', 'why', "won't", 'once', "wasn't", 'with', 'own', 'before', 'd', 'o', 'a', "that'll", 'both', 'am', 'are', 'while', 'this', 'wouldn', 'weren', 'on', 'its', 'itself', 've', 'off', 'ove

In [91]:
def cleanMe(html):
    soup = bs(html,"lxml") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n '.join(chunk for chunk in chunks if chunk)
    text = text.replace('\\n', ' ').replace('\\t', ' ').replace("\n", ' ').replace('\\b', ' ')
    # gets rid of escape characters
    text = bytes(text, "utf-8").decode("unicode_escape")
    # gets rid of links 
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text = " ".join(re.findall(r'\s+[a-zA-Z]+\s+', text))
    # make text lower case
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    # remove stop words
    resultwords  = [word for word in text.split() if word not in stop_words]
    result = ' '.join(resultwords)
    return result

In [93]:
#createCSV('./data/spam_no_stop.csv', './data/Spam.mbox', 'Spam')
#createCSV('./data/inbox_no_stop.csv', './data/Inbox.mbox', 'Ham')

In [27]:
spam1 = pd.read_csv("./data/spam1.csv")
print(spam1.shape)
inbox1 = pd.read_csv("./data/inbox.csv")
print(inbox1.shape[0])


(707, 6)
51112


In [3]:
# inbox concatenated vectors = icv
icv_id = []
icv_word = []
icv_target = []

spam_start = time.time()
for index, row in spam1.iterrows():
    email_words = str(row['Payload']).strip()
    unique_id = row["Message-ID"]
    words_dict = {}
    words_vec = email_words.split(" ")
    for word in words_vec:
        words_dict[word] = True
    for word in words_dict.keys():
        icv_id.append(unique_id)
        icv_word.append(word)
        icv_target.append(1)

print("Spam Time taken: ", str(time.time() - spam_start))

inbox_start = time.time()
for index, row in inbox1.iterrows():
    email_words = str(row['Payload']).strip()
    unique_id = row["Message-ID"]
    words_dict = {}
    words_vec = email_words.split(" ")
    for word in words_vec:
        words_dict[word] = True
    for word in words_dict.keys():
        icv_id.append(unique_id)
        icv_word.append(word)
        icv_target.append(0)

print("Ham Time taken: ", str(time.time() - inbox_start))
            
new_df = pd.DataFrame({"id":icv_id, "word": icv_word, "target": icv_target})


Spam Time taken:  0.10048580169677734
Ham Time taken:  6.080684661865234


In [19]:
word_spam_df = new_df
word_spam_df_features = word_spam_df.loc[:, "word"]

word_spam_df_outcomes = word_spam_df.loc[:, "target"]
word_spam_df.shape
word_spam_df
# tyring to take out stop words after df is made. didn't do 
df = word_spam_df[word_spam_df.word != stop_words]
df.shape

(3850316, 3)

In [22]:
total_ham = inbox1.shape[0]
total_spam = spam1.shape[0]
total = total_spam + total_ham
p_ham = total_spam / total
p_spam = total_ham / total
print(p_ham, p_spam)


0.013643644223161389 0.9863563557768386


In [23]:
def getPriors(word_and_spam, ham_total, spam_total, ham_p, spam_p):
    seen_words = {}
    word_probs = []
    # these could also perhaps be total spam or ham emails?
    for index, row in word_and_spam.iterrows():
        word = row['word']
        if (word not in seen_words):
            seen_words[word] = True
            w_r = word_and_spam[word_and_spam["word"] == word]
            spam_word_count = len(w_r[(word_and_spam["target"] == 1)])
            ham_word_count = len(w_r[(word_and_spam["target"] == 0)])
            p_spam_word = spam_word_count / spam_total
            p_ham_word = ham_word_count / ham_total
            p_s = p_spam_word * spam_p
            p = (p_s)  / (p_s + p_ham_word * ham_p)
            word_probs.append({"key": word, "val": p})
    return word_probs

    
print(getPriors(word_spam_df, total_ham, total_spam, p_ham, p_spam))

C:\Users\maryh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\maryh\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: 

In [ ]:
# didn't get time to write this...but I think the getProirs dictionary could get changed to a matrix easily
#matrixPrior(words_spam):
#    for 

In [28]:
from sklearn import naive_bayes as nb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(word_spam_df_features, word_spam_df_outcomes, random_state=12345, test_size=0.33)


In [26]:
nb_model_1 = nb.GaussianNB().fit(X_train, y_train)
nb_model_1.score(X_test, y_test)

ValueError: could not convert string to float: 'how'

In [ ]:
# parameters for pipeline

# random forest
rfc = RandomForestClassifier()
param_grid_rfc = {'randomforestclassifier__n_estimators': np.arange(1,10)}

# niave bayse
clf = GaussianNB()
param_grid_rfc = {'gaussiannb__prior': True, False}

In [ ]:
def run_model(model, param_grid, xtrain, ytrain, do_pca = False):
    scaler = MinMaxScaler()
    if(do_pca == True):
        pca = PCA(n_components = 10)
        pipe = make_pipeline(pca, model)
    else:
        pipe = make_pipeline(model)
    grid = GridSearchCV(pipe,param_grid)
    grid.fit(xtrain, ytrain)
    grid.best_params_
    accuracy = grid.score(xtrain, ytrain)
    print(f"In-sample accuracy: {accuracy:0.2%}")
    return(grid)

In [ ]:
# add 